In [1]:
import pyqrcode
import pyzbar
import png
from pyqrcode import QRCode
import cv2
import webbrowser
import pandas as pd
import numpy as np
import datetime

In [2]:
def scan_product_qrcode():
    cap = cv2.VideoCapture(0)
    detector = cv2.QRCodeDetector()
    while True:
        _,img=cap.read()
        data,one, _=detector.detectAndDecode(img)
        if data:
            a=data
            break
        cv2.imshow('qrcodescanner app',img)
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return a 

In [3]:
# this needs to be put in a different file and only run once
data = {'Order Number':[1,2,3,4,5,6,7,8,9,10],
     'Order Time':["13:20, 14-2-2021","15:30, 14-2-2021","15:43, 14-2-2021","16:40, 14-2-2021","17:02, 14-2-2021",
                   "19:06, 14-2-2021","19:47, 14-2-2021","20:09, 14-2-2021","20:18, 14-2-2021","21:09, 14-2-2021"],
     'Product Name':["iphone 12","iphone 12 pro","iphone 12 pro","logi keyboardxxx","ipad pro",
                     "macbook pro","logi mousexxx","iphone 12","iphone 12 pro max","iphone 12"],
     'Product Code':["01023","39480","85743","58934","84839","39407","93504","59348","98954","30299"],
     'Buyer':["Matt Iqra","Ansari Mishaal","Appadoo Disha","Bishop Zac","Phil Fred","Bell Johnathan","Bruca Tudor","Mirza Nabeel","Megan Wailly","Muir Ben"],
     'Scheduled Pick Up Time':["slot 3","slot 3","slot 3","slot 3","slot 3","slot 3","slot 7","slot 4","slot 6","slot 8"],
     'Cabin #':['1','2','3','4','5','6','NaN','NaN','NaN','NaN']}
operate_info = pd.DataFrame(data)
operate_info["Actual Pick Up Time"] = np.nan

In [4]:
operate_info

,Order Number,Order Time,Product Name,Product Code,Buyer,Scheduled Pick Up Time,Cabin #,Actual Pick Up Time
0,1,"13:20, 14-2-2021",iphone 12,01023,Matt Iqra,slot 3,1,NaN
1,2,"15:30, 14-2-2021",iphone 12 pro,39480,Ansari Mishaal,slot 3,2,NaN
2,3,"15:43, 14-2-2021",iphone 12 pro,85743,Appadoo Disha,slot 3,3,NaN
3,4,"16:40, 14-2-2021",logi keyboardxxx,58934,Bishop Zac,slot 3,4,NaN
4,5,"17:02, 14-2-2021",ipad pro,84839,Phil Fred,slot 3,5,NaN
5,6,"19:06, 14-2-2021",macbook pro,39407,Bell Johnathan,slot 3,6,NaN
6,7,"19:47, 14-2-2021",logi mousexxx,93504,Bruca Tudor,slot 7,NaN,NaN
7,8,"20:09, 14-2-2021",iphone 12,59348,Mirza Nabeel,slot 4,NaN,NaN
8,9,"20:18, 14-2-2021",iphone 12 pro max,98954,Megan Wailly,slot 6,NaN,NaN
9,10,"21:09, 14-2-2021",iphone 12,30299,Muir Ben,slot 8,NaN,NaN


In [5]:
isarrived = 1
process = 0

In [6]:
def get_order_info():
    print("Please scan your product QR code!")
    decode = scan_product_qrcode()
    product_code = decode.split(": ")[-1]
    return product_code

In [7]:
def match():
    customer_name = input ("Please type in your first name:")
    get_name = operate_info.loc[idx[0]].at["Buyer"]
    if customer_name.lower() == get_name.split(' ')[0].lower():
        return True
    else:
        return False

In [ ]:
#initialize
process = 0
picked = 0

# delivery robot arrived!
if isarrived == 1:
    # ask customer to scan their QR code
    product_code = get_order_info()
    # find the order from system database
    idx = operate_info.index[operate_info['Product Code'] == product_code].tolist()
    
    # check if product info matches with customer name in the system
    get_name = operate_info.loc[idx[0]].at["Buyer"]
    if not idx:
        print("Your order can not be found in the system! Please enter store and ask for help!")
    else:
        if match():
            print("Processing...")
            process = 1
        else:
            t=0
            for i in range (0,3):
                t+=1
                print("Customer name and order don't match !")
                request = input ("Try typing first name again ? (Y/N)")

                if request in ["N","n","no","NO","No"]:
                    print("Thanks. See you.")
                    break
                elif request in ["Y","y","yes","YES","Yes"]:
                    get_name = operate_info.loc[idx[0]].at["Buyer"]
                    if match ():
                        print("Processing...")
                        process = 1
                        break
                    elif t == 3 :
                        print("Exceed maximum trial times. Try again later.")
                        break
                else:
                    print("Invalid input!")
                    break
                    
# if match: process "open cabin"            
if process == 1:
    cabin_num = operate_info.loc[idx[0]].at["Cabin #"]
    # if customer does not come during the scheduled pick up time
    if cabin_num == "NaN":
        print("Your order is not ready for pick up! Please come during scheduled time slot or enter store for help!")
    else:
        print("Opening cabin #" + cabin_num)
        picked = 1
                
# update pick up time into system database
if picked == 1:
    pickup_time = str(datetime.datetime.now()).split(".")[0]
    operate_info.at[idx,'Actual Pick Up Time'] = pickup_time
    print('Item has been picked up by customer at ' + pickup_time)
    

Please scan your product QR code!
Please type in your first name:mingxiang
Customer name and order don't match !
Try typing first name again ? (Y/N)yes


In [27]:
operate_info

,Order Number,Order Time,Product Name,Product Code,Buyer,Scheduled Pick Up Time,Cabin #,Actual Pick Up Time
0,1,"13:20, 14-2-2021",iphone 12,01023,Matt Iqra,slot 3,1,NaN
1,2,"15:30, 14-2-2021",iphone 12 pro,39480,Ansari Mishaal,slot 3,2,NaN
2,3,"15:43, 14-2-2021",iphone 12 pro,85743,Appadoo Disha,slot 3,3,NaN
3,4,"16:40, 14-2-2021",logi keyboardxxx,58934,Bishop Zac,slot 3,4,NaN
4,5,"17:02, 14-2-2021",ipad pro,84839,Phil Fred,slot 3,5,NaN
5,6,"19:06, 14-2-2021",macbook pro,39407,Bell Johnathan,slot 3,6,NaN
6,7,"19:47, 14-2-2021",logi mousexxx,93504,Bruca Tudor,slot 7,NaN,NaN
7,8,"20:09, 14-2-2021",iphone 12,59348,Mirza Nabeel,slot 4,NaN,NaN
8,9,"20:18, 14-2-2021",iphone 12 pro max,98954,Megan Wailly,slot 6,NaN,NaN
9,10,"21:09, 14-2-2021",iphone 12,30299,Muir Ben,slot 8,NaN,NaN
